# Analizando datos del COVID-19 con Pandas

Este es un ejercicio que hago porque una de mis asignaturas pendientes es familiarizarme con Pandas. Más de una vez lo he usado, acabo tirando de buscador y hago lo que necesito, pero nunca he llegado a familiarizarme tanto con la librería como para estar cómodo con ella. 

Así que decidí aprovechar el COVID-19 y manejar sus datos, es más atractivo gestionar datos reales, y hacerlo en este Notebook y si a ti también te sirve ¡genial!

Un mínimo conocimiento de Python y de como funciona el entorno Jupyter sería necesario pero en realidad muy poco :-)

## Cargar y analizardatos rápidamente


Primero cargamos la librería Pandas que nos permitirá analizar y manipular datos.
Una práctica habitual es llamarla `pd` 

Luego definimos en una variable la URL a los datos que vamos a usar.

In [ ]:
import pandas as pd
url = 'https://opendata.ecdc.europa.eu/covid19/casedistribution/csv'

A continuación con el método `read_csv` leemos los datos (desde hace muchas versiones le puedes pasar directamente una URL como estamos haciendo en este caso). Lo ideal sería tener una copia local de los datos pero para simplificar lo dejaremos así.

El método devolverá los datos como un `DataFrame` que es la estructura de datos principal de Pandas, una representación de los datos en filas y columnas, algo parecido a lo que tendrías en una hoja de cálculo si importaras los datos.

In [ ]:
covid = pd.read_csv(url)

Ahora que el contenido de la variable `covid` es un *DataFrame* podemos utilizar el método `head` para ver las primeras filas.

Podemos ver como el *DataFrame* usa el csv para generar los nombres de las columnas, más adelante veremos como referenciarlas o filtrar por ellas.

Podemos ver que automáticamente se añade una primera columna que es el índice.

In [ ]:
covid.head()

Con la propiedad `columns` de un *DataFrame* podemos ver la lista de columnas, en caso de que `head` no nos ayude o querramos tratarla en nuestro programa.

In [ ]:
covid.columns

También podemos estar interesado en la forma que tienen nuestros datos, para ello podemos usar la propiedad `shape` del *DataFrame* y tener el tamaño de los datos en la forma (número de filas, número de columnas)

In [ ]:
covid.shape

Otro método interesante para analizar los datos es `describe` que nos permite sacar algunos datos estadísticos de las columnas numéricas y obtener alguna idea sobre los datos qué estamos manejando, aunque en este caso no son tan interesantes (quiźas nos pueda servir el *count* y *max* de las variables `cases` o `deaths`)

In [ ]:
covid.describe()

## Filtrando y ordenando datos

Genial, pero empieza a ser un poco engorroso ver los datos así ¿no podemos representarlos de alguna forma u ordenarlos aunque sea? Por supuesto :)

Podemos empezar seleccionando sólo las columnas que nos interesan, sólo estamos interesados en las muertes ¿no?

In [ ]:
covid['deaths']

Las columnas pueden ser accedidas como métodos, pero en nuestro caso lo usaremos como índices porque así nos permite gestionar cualquier nombre de columna aunque contenga caracteres no válidos en el nombre de una propiedad (como espacios o -) y porque nos es más fácil añadir varias columnas. Pero por si ves la notación en otros sitios que sepas que lo siguiente es igual y lo comprobamos con el método `equals` que nos permite comparar *DataFrames*

También puede que estemos interesado en ver qué paises están incluidos en el listado usando la función `unique`

In [ ]:
covid['countriesAndTerritories'].unique()

In [ ]:
covid['deaths'].equals(covid.deaths)

Bueno sólo las muertes no, estaríamos interesados también en algunas otras columnas y no hay ningún problema, al índice en vez de una columna le podemos pasar una lista de columnas y nos devolverá un *DataFrame* sólo con esas columnas:

In [ ]:
covid[['dateRep', 'countriesAndTerritories', 'deaths']]

Podemos por ejemplo buscar los días con más mortalidad filtrando y ordenando. Para ello le pasamos a nuestro *DataFrame* `covid` la condición, en este caso buscamos los días con más de 1000 muertos.

In [ ]:
covid[covid['deaths'] > 1000]

¿Y cómo ha funcionado eso? Si vemos estamos pasando el resultado de una comparación como índice. Esa comparación que hacemos `covid['deaths'] > 1000` devuelve una serie de valores *booleanos* que cuando se la pasamos como índice al *DataFrame* nos devuelve sólo el subconjunto de filas que verifican esa condición.

Como tenemos el *DataFrame* quizás querramos ordenarlo, para ello tenemos la función `sort_values`. De esta manera tan simple vemos como es muy sencillo filtrar y ordenar los valores de un *DataFrame* 

In [ ]:
covid[covid['deaths'] > 1000].sort_values('deaths')

Ahora estaréis pensando que el número absoluto de muertes no es un dato tan interesante, depende mucho del número de casos y sobre todo de la población. Como el dato de la población por ahora no lo tenemos en nuestros datos, vamos a calcular el índice de mortalidad en cada país. ¿Cómo haríamos eso? Con dividir las muertes por los casos debería bastar, y en Pandas no necesitas más con un *DataFrame*.

In [ ]:
covid['mortality'] = covid['deaths'] / covid['cases']
covid.head()

Nota: Para evitar unos *warning* al operar con un DataFrame con *NaN*, cambiaremos todos esos *NaN* por 0 usando el método `fillna` y antes de ello todos los *Inf* por *NaN*, pero puesto que *Inf* está definido en numpy (otra librería usada por Pandas) tendremos que importarla y es una práctica habitual importarla como np.

In [ ]:
import numpy as np
covid = covid.replace(np.inf, np.nan).fillna(0)

Ahora que hemos añadido una nueva columna `mortality` vemos como hay valores *NaN* (Not a Number) que vienen de dividir entre 0, lo interesante es que Pandas no ha soltado una excepción lo que nos permite manipular los datos con cierta tranquilidad, pero debemos saber lo que estamos haciendo.

En este caso podéis ver también como a `sort_values` se le pasa el argumento `ascending` para cambiar el orden y hacerlo descendente.

In [ ]:
covid[covid['deaths'] > 1000].sort_values('mortality', ascending=False)

## Agrupando datos

Una función interesante es agrupar los valores de un *DataFrame*, lo que nos devolverá un *DataFrameGroupBy*, bastante similar pero agrupado. Para ello vamos a crear otra variable `covid_by_country` con los datos del *DataFrame* que tenemos en `covid` pero agrupado por el nombre del país (quizás tendría más sentido usar el `geoId` pero por hacer más humana la representación usaré la columna `countriesAndTerritories`)

Si usas `covid_by_country.describe()` podrás ver como la representación cambia con respecto a la descripción que hemos visto abajo, los datos están ordenados por filas en las que cada fila representa un grupo y las columnas están agrupadas en 2 niveles, un primer nivel es el nombre de la columna original y el segundo nivel es el valor estadístico de esa variable para ese grupo. Esta información podría ser más interesante.

In [ ]:
covid_by_country = covid.groupby('countriesAndTerritories')
covid_by_country.describe()

Ya hemos manipulado en filas agrupando, pero tanto un *DataFrame* como un *DataFrameGroupBy* pueden ser manipulados por columnas, por ejemplo puedo interesarme sólo por las muertes:

In [ ]:
covid_by_country['deaths'].describe()

O puedo incluso escoger variaas columnas si quiero, en este caso las variables de `deaths` y `cases` me parecen interesantes.

In [ ]:
covid_by_country[['deaths', 'cases']].describe()

Está bien que describe nos de los datos, pero ¿y si los queremos tener nosotros? Sin problemas, podemos escoger una columna y sumarla de una forma bastante intuitiva (estamos usando la función `sum` pero también tienes disponibles `mean` para la media, `max` para el máximo, `std` para calcular la desviación estándar y muchas más)

Tengamos en cuenta que esas funciones nos devolverán una *Serie* y no un *DataFrame* con lo que los resultados no son tan manipulables.

In [ ]:
covid_by_country['deaths'].sum()

Pero si en vez de tener una *Serie* queremos un *DataFrame* podemos usar el método `agg` al que le decimos que operación queremos hacer en qué columna y nos devolverá un DataFrame con los datos agregados. Ten en cuenta que este nuevo *DataFrame* utilizará el valor de agregación como índice y sólo tendrá como columna el valor agregado.

In [ ]:
deaths_by_country = covid_by_country.agg({'deaths': 'sum'})
deaths_by_country

También podríamos agregar por varios campos, por ejemplo si al final nos interesa la mortalidad podemos usar ese dato.

In [ ]:
mortality_by_country = covid_by_country.agg({'deaths': 'sum', 'cases': 'sum', 'mortality': 'mean'})
mortality_by_country.sort_values('mortality', ascending=False)

## Representando los datos

Es posible cambiar la forma en la que representamos los datos usando el mecanismo de estilo de Pandas. Por ejemplo si queremos representar la mortalidad como % en vez de tanto por uno podríamos hacer crear una función y aplicarla con el método `Style.apply` y restringirlo a la columna de mortalidad.

In [ ]:
mortality_sorted = mortality_by_country.sort_values('mortality', ascending=False)
mortality_sorted.head().style.format({"mortality": "{:.2f}%"})

Algo mejor, pero sólo estamos añadiendo un % al final, no estamos convirtiendo el valor a porcentaje, para ello debemos definir una función que se pasará como argumento para los campos que queremos modificar.

Podriamos ponerla como `lambda` pero lo dejamos como ejercicio para el lector, mi objetivo no es hacer el código más ingenioso

In [ ]:
def as_percent(val):
    return "{:.2f}%".format(val*100.0)

mortality_sorted.head().style.format({"mortality": as_percent})

Otra cosa que podemos hacer es el método `applymap` para aplicar un metodo que devolverá propiedades CSS que se aplicarán al representar los datos. En este ejemplo todo aquella mortalidad por encima del 0.1 por uno (10%) la pintaremos en rojo, pero podemos cambiarlo fácilmente si queréis. Como podéis ver sólo lo aplicaremos a *mortality* usando el argumento `subset`.

In [ ]:
def highlight(val):
    color="red" if val>0.01 else "black"
    return "color: {}".format(color)

mortality_sorted.style.applymap(highlight, subset=['mortality'])

Por supuesto, podemos encadenar las llamadas y así tener representaciones más complejas, si juntamos las 2 modificaciones de estilo que hemos hecho antes tendremos una tabla más bonita.

In [ ]:
mortality_sorted.style.applymap(highlight, subset=['mortality']).format({"mortality": as_percent})

Algo más "complicado" pero que también se puede hacer es representar en verde las mortalidades de los países con un elevado número de casos, ya que lo consideramos más interesante. Para ello usamos la función `apply` que aplica la función a los valores, pero en vez de a cada celda como hace `applymap` arriba lo hace a un conjunto, por defecto lo hará a cada columna con lo que la función recibirá todo el valor de la columna y debe devolver una lista o `DataFrame` con las propiedades CSS para cada campo. Pero si se le pasa el valor `axis=1` la función recibira la fila.

Veamos el código de ejemplo que marcará en verdes los datos "interesantes".

In [ ]:
def mortality_high_cases(row):
    # La función debe devolver un DataFrame o una lista de las mismas
    # dimensiones que la fila
    result = ["" for _ in row.index]
    if row['cases'] > 1000:
        # Llamamos get_loc() para encontrar el índice del campo que 
        # queremos modificar aplicando la propiedad CSS
        result[row.index.get_loc("mortality")] = "color: green"
    return result

mortality_sorted.style.apply(mortality_high_cases, axis=1).format({"mortality": as_percent})

Podemos hacer lo mismo que antes pero además pasar argumentos a la función que estamos llamando.

In [ ]:
def mortality_high_cases(row, limit=1000, color='green'):
    # La función debe devolver un DataFrame o una lista de las mismas
    # dimensiones que la fila
    result = ["" for _ in row.index]
    if row['cases'] > limit:
        # Llamamos get_loc() para encontrar el índice del campo que 
        # queremos modificar aplicando la propiedad CSS
        result[row.index.get_loc("mortality")] = "color: {}".format(color)
    return result

mortality_sorted.style.apply(mortality_high_cases, limit=300, color='blue', axis=1).format({"mortality": as_percent})

En realidad se pueden hacer muchas cosas manipulando los estilos, si quieres aprender más puedes mirar en la documentación oficial: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

### Gráficos

Seamos serios, está guay formatear el texto pero todos hemos visto las gráficas en algún Notebooks ¡y hemos venido a por eso! Así que como estamos usando un Notebook de Jupyter una cosa chula de Pandas es que podemos representar gráficas de nuestros datos. Para ello añadimos la siguiente directiva que incluirá los gráficos en las celdas de nuestro Notebook.

In [ ]:
%matplotlib inline

Ya que tenemos un *DataFrame* que llamamos `deaths_by_country` en el que agregamos los datos vamos a representarlo rápidamente tan sólo llamando al método `plot`

In [ ]:
deaths_by_country.plot()

Podemos ajustar el tamaño de las gráficas por defecto con unas variables de matplotlib (la librería que Pandas usa para crear los gráficos). El tamaño en este caso es ancho y alto en "pulgadas". Esto es opcional pero en mi caso las gráficas serían más bonitas, prueba a ajustar si quieres el tamaño.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [15, 5]

La gráfica de arriba no es demásiado útil, ya que representar los datos de los 204 países no nos aporta muchos pero como ya sabemos filtrar datos y ordenaros, vamos a encadenar varias funciones y llamadas para tener una gráfica interesante. Le pasaremos como argumento `kind='bar'` para que los represente como barras, ya que en realidad no hay ninguna relación entre un punto y el anterior.

In [ ]:
deaths_by_country[deaths_by_country['deaths'] > 500].sort_values('deaths').plot(kind='bar')

O podemos representar las muertes de un país ordenadas por fecha, podemos ver en este caso como ordenamos por varias columnas

In [ ]:
spain_deaths = covid[covid['countriesAndTerritories'] == 'Spain'].sort_values(['year','month','day'])[['dateRep','deaths']]
spain_deaths.plot(kind='area')

Por supuesto, si en vez de usar una columna usamos varias, Pandas se ocupará de representar todos esos datos lo que nos permite mostrar en una gráfica los casos y las muertes.

In [ ]:
spain_death_cases = covid[covid['countriesAndTerritories'] == 'Spain'].sort_values(['year','month','day'])[['dateRep','deaths','cases']]
spain_death_cases.plot(kind='area')

Si en vez de en una sola gráfica queremos tener varias podemos añadir el argumento `subplots=True` a la función `plot`

In [ ]:
spain_death_cases.plot(kind='line', subplots=True)

## Mejorando las gráficas

Ya hemos visto como se puede representar fácilmente los valores que tenemos en la datatable y ahora estaría bien que pudieramos ajustar como se representa. Para ello vamos a usar los datos que tenemos, pero vamos a intentar hacer algo más atractivo. Vamos a utilizar la función `diff` de un *DataFrame* para generar una nueva columna. Esa función devuelve la diferencia de valores de un *DataFrame* con respecto a su columna anterior. Esto ya lo hicimos antes para calcular la columna `mortality` pero en este caso lo haremos con 2 columnas y usando la función `diff`.

También vamos a crear una funcion `death_cases` que acepte un pais y devuelva el DataFrame con la fecha, las muertes, y los casos la diferencia entre casos y muertes. También a diferencia de anteriormente usaremos la función `tail` para ver las últimas filas ya que estamos ordenando por fecha y nos interesa más lo más reciente.

In [ ]:
def death_cases(country='Spain'):
    data = covid[covid['countriesAndTerritories'] == country]\
             .sort_values(['year','month','day'])\
             [['dateRep','deaths','cases']]
    data[['d_deaths', 'd_cases']] = data[['deaths','cases']].diff(1)
    data['mortality'] = data['deaths'] / data['cases']
    return data

death_cases('United_States_of_America').tail()

Ahora que tenemos estos datos podemos empezar con la gráfica base y a partir de ahí ver qué queremos hacer.

In [ ]:
death_cases('United_States_of_America').plot(kind='line')

Una primera modificación a nuestras gráficas es usar una _grid_ para representar las gráficas con más control, no tan sólo usar el argumento `subplots=True` como hemos hecho antes. Esta es una primera aproximación en la que creamos una grid de 2 filas (argumento `nrowss`) y 2 columnas (argumento `ncols`) y luego usamos esos ejes para renderizar las gráficas usando `subplots=True`

Hay que tener en cuenta 2 cosas, rpimero que el número de subplots (`nrows`\*`ncols`) debe ser igual al número de gráficas (los 4 valoes) y que debemos usar `stacked=False` porque por defecto es True pero eso falla al haber valores negativos.

In [ ]:
country = 'United_States_of_America'
data = death_cases(country)
label = "Casos y muertes en {}".format(country)

fig, axes = plt.subplots(nrows=2, ncols=2)
fig.text(x=0.5, y = 1.0, s=label, horizontalalignment='center')

data[['deaths', 'cases', 'd_deaths', 'd_cases']].plot(kind='area', legend=True, \
                                                      stacked=False, subplots=True, ax=axes)

## Por ahora eso es todo

Y por ahora eso es todo. Esto nos debería dar una herramienta para hacer algunos trabajos básicos con algunos datos. Por supuesto quedan muchas cosas que hacer, algunas relacionadas con como afinar los gráficos lo que implica meterse en el apasionante mundo de matplotlib, o también como analizar esos datos, como mezclar varias fuentes de datos (estoy pensando en población de cada país, para sacar datos proporcionales), analizar tendencias y mucho más. 

Pero estos primeros pasos deberían servirnos para tener una idea de cómo usar esta fantástica herramienta que es Python.